In [1]:
import boto3
import urllib3
import os
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create S3 client
s3 = boto3.client(
    "s3",
    endpoint_url="https://swfs-s3.lab.s-miras.com/",
    aws_access_key_id=os.getenv("S3_AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("S3_AWS_SECRET_ACCESS_KEY"),
    verify=False,
)

# List buckets
response = s3.list_buckets()
pprint(response)

{'Buckets': [{'CreationDate': datetime.datetime(2025, 9, 13, 15, 22, 48, tzinfo=tzutc()),
              'Name': 'base'}],
 'Owner': {'ID': ''},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
                                      'content-length': '214',
                                      'content-type': 'application/xml',
                                      'date': 'Tue, 16 Sep 2025 05:10:49 GMT',
                                      'server': 'SeaweedFS 30GB 3.93',
                                      'x-amz-request-id': '1757999449107469152'},
                      'HTTPStatusCode': 200,
                      'HostId': '',
                      'RequestId': '1757999449107469152',
                      'RetryAttempts': 0}}


In [2]:
bucket_name = "airflow"

In [4]:
# Create bucket
try:
    s3.create_bucket(Bucket=bucket_name)
    pprint(f"Bucket '{bucket_name}' created successfully")
except Exception as e:
    pprint(f"Error creating bucket: {e}")

"Bucket 'airflow' created successfully"


In [5]:
# show files in bucket
response = s3.list_objects_v2(Bucket=bucket_name)
pprint(response)

{'EncodingType': 'url',
 'IsTruncated': False,
 'KeyCount': 0,
 'MaxKeys': 10000,
 'Name': 'airflow',
 'Prefix': '',
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
                                      'content-length': '271',
                                      'content-type': 'application/xml',
                                      'date': 'Tue, 16 Sep 2025 05:11:02 GMT',
                                      'server': 'SeaweedFS 30GB 3.93',
                                      'x-amz-request-id': '1757999462018206550'},
                      'HTTPStatusCode': 200,
                      'HostId': '',
                      'RequestId': '1757999462018206550',
                      'RetryAttempts': 0}}


In [6]:
s3.put_object(Bucket=bucket_name, Key="test.txt", Body=b"Hello World")

{'ResponseMetadata': {'RequestId': '1757999464669109814',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'date': 'Tue, 16 Sep 2025 05:11:04 GMT',
   'etag': '"b10a8db164e0754105b7a99be72e3fe5"',
   'server': 'SeaweedFS 30GB 3.93',
   'x-amz-request-id': '1757999464669109814'},
  'RetryAttempts': 0},
 'ETag': '"b10a8db164e0754105b7a99be72e3fe5"'}

In [7]:
# Download file
response = s3.get_object(Bucket=bucket_name, Key="test.txt")
pprint(response["Body"].read())

b'Hello World'


In [8]:
# Configure lifecycle policy to delete objects after 3 days
lifecycle_config = {
    "Rules": [
        {
            "ID": "DeleteAfter3Days",
            "Status": "Enabled",
            "Filter": {"Prefix": ""},  # Apply to all objects
            "Expiration": {"Days": 3},
        }
    ]
}

try:
    s3.put_bucket_lifecycle_configuration(
        Bucket=bucket_name, LifecycleConfiguration=lifecycle_config
    )
    print("Lifecycle policy applied successfully")
except Exception as e:
    print(f"Error applying lifecycle policy: {e}")

Lifecycle policy applied successfully


In [9]:
import json

# Verify lifecycle policy
try:
    response = s3.get_bucket_lifecycle_configuration(Bucket=bucket_name)
    print("Current lifecycle configuration:")
    print(json.dumps(response["Rules"], indent=2))
except Exception as e:
    print(f"Error getting lifecycle policy: {e}")

Current lifecycle configuration:
[
  {
    "Expiration": {
      "Days": 3
    },
    "Prefix": "",
    "Status": "Enabled"
  }
]
